In [1]:
import pandas as pd
import plotly.figure_factory as pf
import numpy as np
import plotly.graph_objects as pg
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.offline as py
from datetime import timedelta

In [2]:
district = pd.read_csv('https://api.covid19india.org/csv/latest/district_wise.csv',index_col='SlNo',skiprows=[1])
state = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise.csv",skiprows=[1])[['State','Confirmed','Recovered','Deaths','Active']]
daily = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise_daily.csv')
district.drop(district.loc[district['State'] == 'State Unassigned'].index,inplace=True)
for i,x in enumerate(district['District']):
	if x == 'Unknown': district['District'].iloc[i] = district['State'].iloc[i]

In [5]:
daily['Date'] = daily['Date'].apply(lambda x: pd.to_datetime(x))

In [7]:
date = daily['Date'].iloc[-1] - timedelta(days=30)

In [8]:
confirmed = daily['TT'].loc[(daily['Date']>=date) & (daily['Status']=='Confirmed')].reset_index(drop=True)

In [9]:
state_names={}
for x in district['State'].unique():
    state_names[x]=district['State_Code'].loc[district['State']==x].values[0]

In [11]:
recovered = daily['TT'].loc[(daily['Date']>=date) & (daily['Status']=='Recovered')].reset_index(drop=True)
deaths = daily['TT'].loc[(daily['Date']>=date) & (daily['Status']=='Deceased')].reset_index(drop=True)

In [12]:
active = confirmed - recovered - deaths

In [13]:
dates = pd.date_range(start=date,end=daily['Date'].iloc[-1])

In [14]:
table_trace1 = pg.Table(header=dict(values=['States', 'Confirmed','Active','Deceased','Recovered']),
                 cells=dict(values=[state['State'], state['Confirmed'],state['Active'], state['Deaths'], state['Recovered']]),domain=dict(x=[0, 0.5],y=[0, 1.0]))
trace1 = pg.Bar(x=dates,y=confirmed,name='Confirmed',marker={'color':'#FD073A'},xaxis='x1',yaxis='y1',textposition='auto',text=confirmed,texttemplate='%{text:.2s}',textangle=90)
trace2 = pg.Bar(x=dates,y=active,name='Active',marker={'color':'#0080FF'},xaxis='x2',yaxis='y2',textposition='auto',text=active,texttemplate='%{text:.2s}',textangle=90)
trace3 = pg.Bar(x=dates,y=recovered,name='Recovered',marker={'color':'#3ADF00'},xaxis='x3',yaxis='y3',textposition='auto',text=recovered,texttemplate='%{text:.2s}',textangle=90)
trace4 = pg.Bar(x=dates,y=deaths,name='Deceased',xaxis='x4',yaxis='y4',textposition='auto',text=deaths,texttemplate='%{text:.2s}',textangle=90,marker={'color':'#FF8000'})

In [15]:
axis=dict(
    showline=False,
    zeroline=False,
    showgrid=False,
    mirror=True,
    ticklen=5,
    gridcolor='#ffffff',
    tickfont=dict(size=10)
)

In [16]:
layout1 = dict(
	autosize=True,
	title='Covid-19 Statistics',
	margin = dict(t=100),
	showlegend=True,
	xaxis1=dict(axis, **dict(domain=[0.55, 1], anchor='y1',showticklabels=False)),
	xaxis2=dict(axis, **dict(domain=[0.55, 1], anchor='y2',showticklabels=False)),
	xaxis3=dict(axis, **dict(domain=[0.55, 1], anchor='y3',showticklabels=False)),
	xaxis4=dict(axis, **dict(domain=[0.55, 1], anchor='y4')),
	yaxis1=dict(axis, **dict(domain=[0.7+0.02, 1.0], anchor='x1')),
	yaxis2=dict(axis, **dict(domain=[0.4+0.07, 0.7], anchor='x2')),
  yaxis3=dict(axis, **dict(domain=[0.2 + 0.02, 0.4+0.05], anchor='x3')),
  yaxis4=dict(axis, **dict(domain=[0.0, 0.2], anchor='x4'))
)

In [17]:
fig1 = dict(data=[table_trace1, trace1, trace2, trace3, trace4], layout=layout1)
py.plot(fig1)

'temp-plot.html'